In [2]:
import json
import torch
import pickle
import numpy as np
import torch.nn as nn
from scipy import stats
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from nn_utils import Attention, EmotionRegression, FeatureExtaction
from generate_embeddings import *

Vocab: torch.Size([19895, 300])


 11%|█         | 897/8062 [00:03<00:24, 297.42it/s]


ValueError: 'hikesthey' is not in list

In [2]:
batch_size = 64
lr = 1e-4
epochs = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [3]:
# dict = {}
# with open("dataset.pkl", "rb") as fin :
#     dict = pickle.load(fin)

# vocab = dict['vocab'].to(device)
# vocab.requires_grad = False
# train_dataloader = DataLoader(
#     dict['train_dataset'],
#     shuffle=True,
#     batch_size=batch_size
#     )
# val_dataloader = DataLoader(
#     dict['dev_dataset'],
#     shuffle=False,
#     batch_size=batch_size
#     )
dataset = Dataset()
vocab = dataset.get_vocab().to(device)
train_dataset = dataset.read_data('./data/train.csv') 
dev_dataset = dataset.read_data('./data/dev.csv')
test_dataset = dataset.read_data('./data/test.csv')

In [4]:
class AAN(nn.Module) :
    def __init__(self, embed_size=300, hidden_size=150) :
        #['V', 'A', 'D', 'S']
        super(AAN, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.feature_size = self.hidden_size*4

        self.attention_1 = Attention(self.embed_size)
        self.attention_2 = Attention(self.embed_size)
        self.attention_s = Attention(self.embed_size)

        self.features = FeatureExtaction(self.embed_size, self.hidden_size)
        
        self.regression_1 = EmotionRegression(self.feature_size * 2)
        self.regression_2 = EmotionRegression(self.feature_size * 2)
    
    def forward(self, vocab, sentences, source_lengths) :
        sentences = vocab(sentences)
#         sentences = sentences.detach()

        sentences_1 = self.attention_1(sentences, source_lengths)
        sentences_2 = self.attention_2(sentences, source_lengths)
        sentences_s = self.attention_s(sentences, source_lengths)

        features_1 = self.features(sentences_1, source_lengths)
        features_2 = self.features(sentences_2, source_lengths)
        features_s = self.features(sentences_s, source_lengths)

        value_1 = self.regression_1(torch.cat((features_1, features_s), dim=1))
        value_2 = self.regression_2(torch.cat((features_2, features_s), dim=1))
        output = torch.cat((value_1, value_2), dim=1)
        return output


In [13]:
class Train() :
    def __init__(self, type=[0,1]) :
        super(Train, self).__init__()
        self.type = type
        self.model = AAN()
        self.model.cuda()
#         self.init_weights()
        self.mse = nn.MSELoss()
        self.optim = optim.Adam(self.model.parameters(), lr=lr)
        self.training_stats = []
    
#     def init_weights(self) :
#         for param in self.model.parameters() :
#             temp = np.sqrt(6.0/(sum([i for i in param.shape])+1e-8))
#             param.data.uniform_(-temp, temp)
    
    def run_model(self, batch) :
        sentences = batch[0].to(device)
        source_lengths = batch[1].to(device)
        return self.model(vocab, sentences, source_lengths)
    
#     def get_r(self, output, target) :
#         temp = [(stats.pearsonr(output[:,i].cpu().detach(), target[:,i].cpu().detach())) for i in range(2)]
#         return temp
    
    def train(self, epochs=epochs) :
        mse = nn.MSELoss()
        for epoch_i in range(epochs) :
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            self.model.train()
#             train_r = []
#             train_loss = []
            for i, batch in enumerate(train_dataloader) :

                self.optim.zero_grad()
                target = batch[2][:,self.type].float().to(device)
                output = self.run_model(batch)
                loss = mse(output, target)
#                 train_loss.append(reg_loss.item())
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

                self.optim.step()
                if i%10 == 0 :
                    print("Batch: {} train Loss: {} ".format(i, loss))

#             val_loss = []
#             val_r = []
            for i, batch in enumerate(val_dataloader) :
                with torch.no_grad() :
                    output, p1, p2, target = self.run_model(batch)

                    loss = mse(output, target)
#                     val_loss.append(loss.item())
#                     val_r.append(self.get_r(output, target))
                    if i%10 == 0 :
                        print("Batch: {} val Loss: {} ".format(i, loss))

            self.training_stats.append({
                'training loss' : sum(train_loss)/len(train_loss),
                'validation loss' : sum(val_loss)/len(val_loss),
#                 'train r' : torch.tensor(train_r).mean(dim=0),
#                 'val r' : torch.tensor(val_r).mean(dim=0),
            })
            print(json.dumps(self.training_stats[-1], indent=4))
            
        def save_model(self) :
            #TODO: Add function to save model
            pass
        
        def plot(self, r_values) :
            #TODO: Plot r values
            pass

In [14]:
train = Train()
train.train(10)

======== Epoch 1 / 10 ========
Batch: 0 train Loss: 9.008907318115234 
Batch: 10 train Loss: nan 
Batch: 20 train Loss: nan 


KeyboardInterrupt: 